About the Dataset
The Student_performance_10k.csv dataset includes information about student demographics, parental education, lunch types, test preparation, and scores in math, reading, writing, and science. It also has a calculated total score and a final grade. 


In [1]:
import pandas as pd
df = pd.read_csv("Student_performance_10k.csv")
print(df.head(10))

  roll_no gender race_ethnicity parental_level_of_education  lunch  \
0  std-01   male        group D                some college    1.0   
1  std-02   male        group B                 high school    1.0   
2  std-03   male        group C             master's degree    1.0   
3  std-04   male        group D                some college    1.0   
4  std-05   male        group C                some college    0.0   
5  std-06   male        group B                 high school    1.0   
6  std-07    NaN        group C                some college    0.0   
7  std-08   male        group B          associate's degree    1.0   
8  std-09   male        group C                         NaN    1.0   
9  std-10   male        group C            some high school    NaN   

   test_preparation_course math_score  reading_score  writing_score  \
0                      1.0         89           38.0           85.0   
1                      0.0         65          100.0           67.0   
2               

Adapting the Library System Code
Data Loading: The table_creation_and_insert function will be modified to read the student data CSV and insert it into the MySQL table. 
Functions: The following operations relevenant to student performanace data could be performed:
- Listing students and their score
- Searching for students by name, group, or other criteria
- Calculating average score
- Adding new student records

In [2]:
#Anna: This is part of the code resembles the sample project. I need some time to dive into it.
# Install the relevant libraries
! pip install mysql-connector-python --quiet
! pip install pandas --quiet
! pip install SQlAlchemy --quiet

# import libraries
import mysql.connector
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError

#Global variables
#Import Credentials
host = "localhost"
user = "root"
password = "Nz8iu-zew8jom" # I changed mysql's password to "Password" for security concern as this file is public. 
database = "db_student_performance"
table_name = "tbl_student_performance"
#Anna: Refer to your note why use forward slashes.
csv_path = "C:/Users/annah/Downloads/Student_performance_10k.csv"

In [3]:
#Initialize Database
def database_init(h, u, p, db=None):
    """
    If db is None, returns a connection without selecting a database. 
    Otherswise connects to the specific database. 
    """
    if db: 
        return mysql.connector.connect(host=h, user=u, password=p, database=db)
    else:
        return mysql.connector.connect(host=h, user=u, password=p)
#Create a database if it does not exist
def ensure_database_exists():
    conn = database_init(host, user, password)
    cursor = conn.cursor()
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS `{database}`")
    conn.commit()
    cursor.close()
    conn.close()

#Create a table and insert bulk data
def table_creation_and_insert():
    #Read the CSV and bulk-insert into MySQL using pandas.to_sql
    #1 ensure the database exists
    ensure_database_exists()
    #2 read csv into pandas
    df_students = pd.read_csv(csv_path)
    
    #Anna: I need to revisit this part of the code. I am not familiar with it.
    #3 build the SQLAlchemy engine (no port in URI)
    engine = create_engine(
        f"mysql+mysqlconnector://{user}:{password}@{host}/{database}",
        echo=False
    )
    try:
    #4 Write to SQL replacing the table if it exists
        df_students.to_sql(
            name=table_name, 
            con=engine,
            if_exists="fail",    #fail if table exists
            index=False,
            chunksize=500
        )
        print("Data inserted successfully!")
    except SQLAlchemyError as e:
        print("Failed to insert data:", e)
    finally:
        #5 clean up
        engine.dispose()

    

In [4]:
def list_students():
    conn = database_init(host, user, password, database)
    cursor = conn.cursor()
    cursor.execute(f"SELECT * FROM {table_name}")
    headers = [col[0] for col in cursor.description]
    students = cursor.fetchall()
    
    print("\nStudnet Records:")
    print(headers)
    for s in students: 
        print(s)
    cursor.close()
    conn.close()


In [5]:
def search_students(keyword):
    conn = database_init(host, user, password, database)
    cursor = conn.cursor()
    query = f"SELECT * FROM {table_name} WHERE roll_no LIKE %s OR gender LIKE %s OR race_ethnicity LIKE %s OR parental_level_of_education LIKE %s"
    param = f"%{keyword}%"
    cursor.execute(query, (param, param, param, param))
    results = cursor.fetchall()
    headers = [col[0] for col in cursor.description]
    
    if results:
        print(f"\nSearch Results for '{keyword}'.")
        for row in results: # added this to print the results
            print(row)
    else:
        print(f"No students found matching '{keyword}'.")
    cursor.close()
    conn.close()

In [6]:
def calculate_average_score():
    conn = database_init(host, user, password, database)
    cursor = conn.cursor()
    query = f"SELECT AVG(math_score), AVG(reading_score), AVG(writing_score), AVG(science_score), AVG(total_score) FROM {table_name}"
    cursor.execute(query)
    average_score = cursor.fetchone()
    
    if average_score:
        print("n\Average Scores:")
        print(f" Math: {average_score[0]:.2f}")
        print(f" Reading: {average_score[1]:.2f}")
        print(f" Writing: {average_score[2]:.2f}")
        print(f" Science: {average_score[3]:.2f}")
        print(f" Total: {average_score[4]:.2f}")
    else:
        print("Could not calculate average scores.")
    cursor.close()
    conn.close()

In [7]:
def add_student():
    conn = database_init(host, user, password, database)
    cursor = conn.cursor()
    
    roll_no = input("Enter Roll Number:")
    gender = input("Enter Gender: ")
    race_ethnicity = input("Enter Race/Ethnicity: ")
    parental_level_of_education = input("Enter Parental level of Education: ")
    lunch = int(input("Enter Lunch (0 for standard, 1 for free/ reduced): "))
    test_preparation_course = int(input("Enter Test Preparation Course (0 for none, 1 for completed): "))
    math_score = int(input("Enter Math Score: "))
    reading_score = int(input("Enter Reading Score: "))
    writing_score = int(input("Enter Writing Score: "))
    science_score = int(input("Enter Science Score: "))
    
    total_score = math_score + reading_score + writing_score + science_score
    
    grade = input("Enter Grade: ")
    
    
    #Anna: revisit
    try: 
        cursor.execute(
            f"INSERT INTO {table_name} (roll_no, gender, race_ethnicity, parental_level_of_education, lunch, test_preparation_course, math_score, reading_score, writing_score, science_score, total_score, grade) VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s)",
            (roll_no, gender, race_ethnicity, parental_level_of_education, lunch, test_preparation_course, math_score, reading_score, writing_score, science_score, total_score, grade)
        )
        conn.commit()
        print("Student added successfully.")
    except mysql.connector.Error as err:
        print(f"Failed to add student: {err}")
    finally:
        conn.close()


In [8]:
def main():
    try:
        table_creation_and_insert()
    except:
        print("Table is already created. To create a new table, delete the existing first.")
    while True: 
        print("\nStudent Performance Management System")
        print("1. List all students")
        print("2. Search students")
        print("3. Calculate average scores")
        print("4. Add a new student")
        print("5. Exit")
        
        choice = input("Enter choice: ")
        
        if choice == '1':
            list_students()
        elif choice == '2':
            kw = input("Enter search keyword (roll_no, gender, race_ethnicity, parental_level_education: ")
            search_students(kw)
        elif choice == '3':
            calculate_average_score()
        elif choice == '4':
            add_student()
        elif choice == '5':
            print("Goodbye!")
            break
        else: 
            print("Invalid choice. Please try again.")


In [9]:
main()

Table is already created. To create a new table, delete the existing first.

Student Performance Management System
1. List all students
2. Search students
3. Calculate average scores
4. Add a new student
5. Exit
Enter choice: 2
Enter search keyword (roll_no, gender, race_ethnicity, parental_level_education: std-01

Search Results for 'std-01'.
('std-01', 'male', 'group D', 'some college', 1.0, 1.0, '89', 38.0, 85.0, 26.0, 238.0, 'C')

Student Performance Management System
1. List all students
2. Search students
3. Calculate average scores
4. Add a new student
5. Exit
Enter choice: 4
Enter Roll Number:std-22830
Enter Gender: male
Enter Race/Ethnicity: group A
Enter Parental level of Education: high school
Enter Lunch (0 for standard, 1 for free/ reduced): 1
Enter Test Preparation Course (0 for none, 1 for completed): 1
Enter Math Score: 89
Enter Reading Score: 38
Enter Writing Score: 85
Enter Science Score: 26
Enter Grade: A
Student added successfully.

Student Performance Management Sy